# SVM и его ядра
__Суммарное количество баллов: 10__

# Задание 1 (7 баллов)

В этом задании вам не нужно будет реализовать SVM, а научиться его применять.

Примените метод SVM (например, из библиотеки sklearn) для датасета blobs2.

Визуализируйте результат (разбиение плоскости и опорные вектора) при разных вариантах ядер (линейное; полиномиальное степеней 2,3,5; RBF).

In [ ]:
from sklearn.svm import SVC 
from sklearn.datasets import make_blobs
import numpy as np
from numpy import linalg as LA
import copy
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification, make_moons, make_blobs
from typing import NoReturn, Callable


X_blobs2, y_blobs2  = make_blobs(n_samples=1000, random_state=4, center_box=(-2, 2),cluster_std=0.4, centers=2)

In [ ]:
def visualize(clf, X, y):
    x_min, x_max = X[:, 0].min(), X[:, 0].max()
    y_min, y_max = X[:, 1].min(), X[:, 1].max()
    x_border = (x_max - x_min) / 20 + 1.0e-3
    x_h = (x_max - x_min + 2 * x_border) / 200
    y_border = (y_max - y_min) / 20 + 1.0e-3
    y_h = (y_max - y_min + 2 * y_border) / 200
    
    cm = plt.cm.Spectral

    xx, yy = np.meshgrid(np.arange(x_min - x_border, x_max + x_border, x_h), np.arange(y_min - y_border, y_max + y_border, y_h))
    mesh = np.c_[xx.ravel(), yy.ravel()]

    z_class = clf.predict(mesh).reshape(xx.shape)

    # Put the result into a color plot
    plt.figure(1, figsize=(8, 8))
    plt.pcolormesh(xx, yy, z_class, cmap=cm, alpha=0.3, shading='gouraud')

    # Plot hyperplane and margin
    z_dist = clf.decision_function(mesh).reshape(xx.shape)
    plt.contour(xx, yy, z_dist, [0.0], colors='black')
    plt.contour(xx, yy, z_dist, [-1.0, 1.0], colors='black', linestyles='dashed')

    # Plot also the training points
    y_pred = clf.predict(X)

    ind_support = []
    ind_correct = []
    ind_incorrect = []
    for i in range(len(y)):
        if i in clf.support_:
            ind_support.append(i)
        elif y[i] == y_pred[i]:
            ind_correct.append(i)
        else:
            ind_incorrect.append(i)

    plt.scatter(X[ind_correct, 0], X[ind_correct, 1], c=y[ind_correct], cmap=cm, alpha=1., edgecolor='black', linewidth=.8)
    plt.scatter(X[ind_incorrect, 0], X[ind_incorrect, 1], c=y[ind_incorrect], cmap=cm, alpha=1., marker='*',
               s=50, edgecolor='black', linewidth=.8)
    plt.scatter(X[ind_support, 0], X[ind_support, 1], c=y[ind_support], cmap=cm, alpha=1., edgecolor='yellow', linewidths=1.,
               s=40)

    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.tight_layout()

In [ ]:
def task1(X, y, kernel, degree=3):
    svm = SVC(kernel=kernel, degree=degree)
    svm.fit(X, y)
    visualize(svm, X, y)
    plt.show()

In [ ]:
task1(X_blobs2, y_blobs2, kernel='linear')

In [ ]:
task1(X_blobs2, y_blobs2, kernel='poly', degree=2)

In [ ]:
task1(X_blobs2, y_blobs2, kernel='poly', degree=3)

In [ ]:
task1(X_blobs2, y_blobs2, kernel='poly', degree=5)

In [ ]:
task1(X_blobs2, y_blobs2, kernel='rbf')

# Задание 2 (3 балла)

Протестируйте SVM на датасете spam. Для этой задачи используйте данные x_spam_train и y_spam_train:

1. Выполните загрузку и предобработку файлов x_spam_train и y_spam_train.
2. Разбейте x_spam_train и y_spam_train на x_train, y_train, x_test и y_test для оценки точности работы алгоритма.
3. Посчитайте метрики precision, recall, accuracy для модели SVM. Если необходимо, попробуйте разные наборы параметров для получения лучшего результата.
4. Сравните время работы SVM и Random Forest при близкой точности.

Ответьте на следующие вопросы:
1. Какой нужен препроцессинг данных для моделей?
2. Какая модель делает предсказания лучше? Предположите, почему.

1. `x_spam_train/x_spam_test` на этапе препроцессинга стоило удалить абсолютно неинформативный признак Id. 
2. Модель `Random Forest` делает предсказание лучше. Это связано, вероятно, с тем, что векторы, которые соответствуют 0 и 1, нельзя 'удачно' разделить гиперплоскостью 

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
def read_datasets_spam(path_to_x, path_to_y):
    X = read_dataset_X_spam(path_to_x)
    y_df = pd.read_csv(path_to_y, sep=',')
    y = np.array(y_df['Expected'])
    shuffle_idx = np.arange(y.shape[0])
    np.random.shuffle(shuffle_idx)
    
    X = X[shuffle_idx]
    y = y[shuffle_idx]
    return X, y

In [ ]:
def read_dataset_X_spam(path_to_csv):
    df = pd.read_csv(path_to_csv, sep=',')
    return np.array(df.drop('Id', 1))

In [ ]:
path_to_spam_X = '../input/homework-svm-ib-22/x_spam_train.csv'
path_to_spam_y = '../input/homework-svm-ib-22/y_spam_train.csv'

In [ ]:
X_train, y_train = read_datasets_spam(path_to_spam_X, path_to_spam_y)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
def print_metrics(y_test, pred):
    print(f"Accuracy: {accuracy_score(y_test, pred)}")
    print(f"Precision: {precision_score(y_test, pred, pos_label=1)}")
    print(f"Recall: {recall_score(y_test, pred, pos_label=1)}")
    print(f"F1-score: {f1_score(y_test, pred, pos_label=1)}")

### Random forest

In [ ]:
rfc = RandomForestClassifier(max_depth=5, n_estimators=30)

In [ ]:
%timeit rfc.fit(X_train, y_train)

In [ ]:
print_metrics(y_test, rfc.predict(X_test))

### SVM

In [ ]:
svm = SVC(kernel='rbf', gamma='auto')

In [ ]:
%timeit svm.fit(X_train, y_train)

In [ ]:
print_metrics(y_test, svm.predict(X_test))

Обучите модель на всех данных из x_spam_train и y_spam_train.
Сделайте submit своего решения и получите значение f1_score не менее 0.6

In [ ]:
X_train, y_train = read_datasets_spam(path_to_spam_X, path_to_spam_y)
path_to_X_test = '../input/homework-svm-ib-22/x_spam_test.csv'
X_test = read_dataset_X_spam(path_to_X_test)

In [ ]:
svm = SVC(kernel='rbf', gamma='auto')
svm.fit(X_train, y_train)

In [ ]:
submission = pd.DataFrame(columns = ["Id", "Expected"])
submission["Id"] = np.arange(X_test.shape[0])
submission["Expected"] = svm.predict(X_test)
submission.to_csv('submission.csv', index=False)